In [1]:
'''

啟動伺服器基本樣版

設定業務功能
    當用戶關注時
        記錄個資，綁定菜單，回傳文字消息與圖片消息，介紹功能與使用方法
    當用戶點擊自定義菜單或輸入文字消息時
        傳送圖片消息或相應文字消息。
    當用戶點擊按鈕消息的時候
        會將用戶個資紀錄回檔案內，以供未來聯繫用

'''

'\n\n啟動伺服器基本樣版\n\n設定業務功能\n    當用戶關注時\n        記錄個資，綁定菜單，回傳文字消息與圖片消息，介紹功能與使用方法\n    當用戶點擊自定義菜單或輸入文字消息時\n        傳送圖片消息或相應文字消息。\n    當用戶點擊按鈕消息的時候\n        會將用戶個資紀錄回檔案內，以供未來聯繫用\n\n'

In [2]:
"""

啟用伺服器基本樣板

"""
# 引用sql資訊
from db_information import sql_password,sql_user,cloud_ip
# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [3]:
"""


當用戶關注Line@後，Line會發一個FollowEvent，

我們接受到之後，取得用戶個資，對用戶綁定自定義菜單，會回傳消息給用戶


"""
# 引用sql資訊
from db_information import sql_password,sql_user,cloud_ip
# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 載入requests套件
import requests

import pymysql

# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("../users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
    # 用戶資訊存入資料庫
    # 打开数据库连接
    conn = pymysql.connect(host=cloud_ip,
                             port=3306,
                             user=sql_user,
                             password=sql_password,
                             db='news',
                             charset='utf8')
 
    # 使用 cursor() 方法创建一个游标对象 cursor
    cursor = conn.cursor()
 
    # 使用 execute()  方法执行 SQL 
    cursor.execute("REPLACE INTO user SET userid=%s, name=%s", 
                        (str(user_profile.user_id), 
                        str(user_profile.display_name))
                  )
    conn.commit()
    conn.close()
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    
    # 消息清單
    reply_message_list = [
    TextSendMessage(text="歡迎使用嗑新聞"),
    TextSendMessage(text="了解國際，放眼未來，從觀看國際新聞開始。\n每日嗑點新聞，將有助於提升國際觀。")
    ]

    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

<font face="黑" size=5 color="#000000" >關鍵字搜尋

In [4]:
# 關鍵字模板
from linebot.models import(
    TextSendMessage,ButtonsTemplate,PostbackAction,MessageAction,URIAction,TemplateSendMessage
)
from liff_api import *
# initiate token
def keywd(userid):
    secretFileContentJson=json.load(open("../line_secret_key",'r'))
    liff_init(secretFileContentJson.get("channel_access_token"))
    liffUrl = 'https://%s/search/%s'%(str(server_url),str(userid))
    liffID = (liff_add(liffUrl, "compact"))['liffId']

    buttons_template_message = TemplateSendMessage(
        alt_text='Buttons template',
        template=ButtonsTemplate(
            title='關鍵字',
            text='搜尋',
            actions=[
                URIAction(
                    label='點我搜尋',
                    uri="line://app/%s"%(str(liffID))
                )
            ]
        )
    )
    return buttons_template_message

In [ ]:
#關鍵字搜尋呈現
@app.route('/search/<userid>', methods = ['GET'])
def search(userid):
    html="""
    <!doctype html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>This is Liff keyword-search</title>

    <style>
        input[type="submit"]{padding:5px 15px; background:#ccc; border:0 none;
    cursor:pointer;
    -webkit-border-radius: 5px;
    border-radius: 5px;
    width:90px;height:50px;font-size:30px;}

        input[type="text"]{padding:5px 15px; border:2px black solid;
    cursor:pointer;
    -webkit-border-radius: 5px;
    border-radius: 5px; 
    width:300px;height:50px;font-size:30px;}

    .demo{
        padding-top: 50px;
        text-align: center
    }
    </style>

    </head>
    <body>
    
    <script type="text/javascript">
    function ValidateValue(textbox) {
        var IllegalString = "[`~!#$^&*()=|{}':;',\\[\\].<>/?~！#￥……&*（）——|{}【】‘；：”“'。，、？]‘'";
        var textboxvalue = textbox.value;
        var index = textboxvalue.length - 1;
        var s = textbox.value.charAt(index);
        if (IllegalString.indexOf(s) >= 0) {
        s = textboxvalue.substring(0, index);
        textbox.value = s;
        }
    }
    </script>
    <div class="demo">
       <form action="/search/%s" method="post">
        <input type="text" name="keyword" onkeyup = "ValidateValue(this)" placeholder="關鍵字搜尋"style="width:200px;height:50px;font-size:25px" >
        <input type="submit" value="確認" style="width:80px;height:50px;font-size:30px">
       </form> 
    </div>
    </body>
    </html>"""%(userid)
    return html

@app.route('/search/<userid>', methods = ['POST'])
def s_result(userid):
    keyword = request.form.get('keyword')
    conn = pymysql.connect(host=cloud_ip,
                             port=3306,
                             user=sql_user,
                             password=sql_password,
                             db='news_db',
                             charset='utf8')
    # 使用 cursor() 方法创建一个游标对象 cursor
    cursor = conn.cursor()
    # 使用 execute()  方法执行 SQL 查询
    cursor.execute('SELECT url,title\
                    FROM apple_news\
                    WHERE title like "%s" \
                    ORDER BY RAND() \
                    LIMIT 10'%(str("%"+keyword+"%"))
                   )
    # 使用 fetchmany(n)方法获取n条数据.
    data = cursor.fetchmany(10)
    conn.close()
    result = "<title>搜尋%s結果</title>"%(keyword)
    for i in data:
        url = i[0]
        title = i[1]
        link = """
           <div class ="box">
               •<a href="%s" style="font-size:35;text-decoration:none;">%s</a>
           </div>
           """ % (url, title)
        result += link
    return result

<font face="黑" size=5 color="#000000" >關聯字字典

In [ ]:
#關聯字字典模板
from linebot.models import(
    TextSendMessage,ButtonsTemplate,PostbackAction,MessageAction,URIAction,TemplateSendMessage
)
from liff_api import *
# initiate token
def relevance(userid):
    secretFileContentJson=json.load(open("../line_secret_key",'r'))
    liff_init(secretFileContentJson.get("channel_access_token"))
    liffUrl = 'https://%s/relevance/%s'%(str(server_url),str(userid))
    liffID = (liff_add(liffUrl, "compact"))['liffId']

    buttons_template_message = TemplateSendMessage(
        alt_text='Buttons template',
        template=ButtonsTemplate(
            title='關聯字字典',
            text='關聯字',
            actions=[
                URIAction(
                    label='點我搜尋',
                    uri="line://app/%s"%(str(liffID))
                )
            ]
        )
    )
    return buttons_template_message
#關聯字搜尋呈現
@app.route('/relevance/<userid>', methods = ['GET'])
def relevance_search(userid):
    html="""
    <!doctype html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>關聯字字典</title>

    <style>
        input[type="submit"]{padding:5px 15px; background:#ccc; border:0 none;
    cursor:pointer;
    -webkit-border-radius: 5px;
    border-radius: 5px;
    width:90px;height:50px;font-size:30px;}

        input[type="text"]{padding:5px 15px; border:2px black solid;
    cursor:pointer;
    -webkit-border-radius: 5px;
    border-radius: 5px; 
    width:300px;height:50px;font-size:30px;}

    .demo{
        padding-top: 50px;
        text-align: center
    }
    </style>

    </head>
    <body>
    
    <script type="text/javascript">
    function ValidateValue(textbox) {
        var IllegalString = "[`~!#$^&*()=|{}':;',\\[\\].<>/?~！#￥……&*（）——|{}【】‘；：”“'。，、？]‘'";
        var textboxvalue = textbox.value;
        var index = textboxvalue.length - 1;
        var s = textbox.value.charAt(index);
        if (IllegalString.indexOf(s) >= 0) {
        s = textboxvalue.substring(0, index);
        textbox.value = s;
        }
    }
    </script>
    <div class="demo">
       <form action="/relevance/%s" method="post">
        <input type="text" name="keyword" onkeyup = "ValidateValue(this)" placeholder="關聯字搜尋"style="width:200px;height:50px;font-size:25px" >
        <input type="submit" value="確認" style="width:80px;height:50px;font-size:30px">
       </form> 
    </div>
    </body>
    </html>"""%(userid)
    return html

@app.route('/relevance/<userid>', methods = ['POST'])
def relevance_result(userid):
    keyword = request.form.get('keyword')
    conn = pymysql.connect(host=cloud_ip,
                             port=3306,
                             user=sql_user,
                             password=sql_password,
                             db='news',
                             charset='utf8')
    # 使用 cursor() 方法创建一个游标对象 cursor
    cursor = conn.cursor()
    # 使用 execute()  方法执行 SQL 查询
    cursor.execute('SELECT *\
                FROM related_words\
                WHERE keyword like "%s"' %(str(keyword))
                   )
    # 使用 fetchmany(n)方法获取n条数据.
    data = cursor.fetchall()
    conn.close()
    related_text="<title>%s的關聯字</title>"%(keyword)
    try:
        for related in data[0]:
            related_text+="""
            <span style="font-size:1.2cm;">%s</span><br>
            """%(related)
        return related_text
    except IndexError:
        return "not found"

In [ ]:
# 國外推薦 seemode
@app.route('/recommend/seemode', methods = ['GET'])
def seemode():
    conn = pymysql.connect(host=cloud_ip,
                             port=3306,
                             user=sql_user,
                             password=sql_password,
                             db='news',
                             charset='utf8')

    # 使用 cursor() 方法创建一个游标对象 cursor
    cursor = conn.cursor()

    # 使用 execute()  方法执行 SQL 查询
    cursor.execute("SELECT news_title,news_link,img_link \
                   from hot_foreign_news \
                   ORDER BY RAND() \
                   LIMIT 25"
               )
    # 使用 fetchmany(n)方法获取n条数据.
    data = cursor.fetchmany(25)
    conn.close()
    more = """
    <title>just see more</title>
    """

    for i in data:
        img = i[2]
        title = i[0]
        url = i[1]
        link="""
        <div class ="box">
            <img src="%s" width="150" height="150"><a href="%s" style="font-size:20;text-decoration:none;">%s</a>
        </div>
        """%(img,url,title)
        more+=link
    return more

<font face="黑" size=5 color="#000000" >訂閱功能

In [ ]:
# 訂閱模板
import pymysql
from linebot.models import(
    TextSendMessage,ButtonsTemplate,PostbackAction,MessageAction,URIAction,TemplateSendMessage
)
from liff_api import *
# initiate token
def subscription(userid):
    secretFileContentJson=json.load(open("../line_secret_key",'r'))
    liff_init(secretFileContentJson.get("channel_access_token"))
    liffUrl = 'https://%s/subscription/%s'%(str(server_url),str(userid))
    liffID = (liff_add(liffUrl, "compact"))['liffId']

    buttons_template_message = TemplateSendMessage(
        alt_text='Buttons template',
        template=ButtonsTemplate(
            title='訂閱',
            text='推薦訂閱',
            actions=[
                URIAction(
                    label='點我訂閱',
                    uri="line://app/%s"%(str(liffID))
                )
            ]
        )
    )
    return buttons_template_message

'''

訂閱頁面

'''
@app.route('/subscription/<userid>', methods = ['GET'])
def user_subscription(userid):
    html="""
    <!doctype html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>This is Liff subscription</title>

    <style>
        input[type="submit"]{padding:5px 15px; background:#ccc; border:0 none;
    cursor:pointer;
    -webkit-border-radius: 5px;
    border-radius: 5px;
    width:90px;height:50px;font-size:30px;}

        input[type="text"]{padding:5px 15px; border:2px black solid;
    cursor:pointer;
    -webkit-border-radius: 5px;
    border-radius: 5px; 
    width:300px;height:50px;font-size:30px;}

    .demo{
        padding-top: 50px;
        text-align: center
    }
    </style>

    </head>
    <body>
    
    <script type="text/javascript">
    function ValidateValue(textbox) {
        var IllegalString = "[`~!#$^&*()=|{}':;',\\[\\].<>/?~！#￥……&*（）——|{}【】‘；：”“'。，、？]‘'";
        var textboxvalue = textbox.value;
        var index = textboxvalue.length - 1;
        var s = textbox.value.charAt(index);
        if (IllegalString.indexOf(s) >= 0) {
        s = textboxvalue.substring(0, index);
        textbox.value = s;
        }
    }
    </script>
    <div class="demo">
       <form action="/subscription/%s" method="post">
        <input type="text" name="subscription" onkeyup = "ValidateValue(this)" placeholder="訂閱字"style="width:200px;height:50px;font-size:25px" >
        <input type="submit" value="訂閱" style="width:80px;height:50px;font-size:30px">
       </form> 
    </div>
    <div class="demo">
       <form action="/subscription/search/%s" method="get">
        <input type="submit" value="查詢" style="width:80px;height:50px;font-size:30px">
       </form> 
    </div>
    </body>
    </html>"""%(userid,userid)
    return html
@app.route('/subscription/<userid>', methods = ['POST'])
def sub_result(userid):
    subscription = request.form.get('subscription')
    conn = pymysql.connect(host=cloud_ip,
                             port=3306,
                             user=sql_user,
                             password=sql_password,
                             db='news',
                             charset='utf8')
 
    # 使用 cursor() 方法创建一个游标对象 cursor
    cursor = conn.cursor()
 
    # 使用 execute()  記錄使用者訂閱
    cursor.execute("REPLACE INTO user_subscription SET userid=%s, keyword=%s", 
                        (str(userid), 
                        str(subscription))
                  )
    conn.commit()
    conn.close()
    sub="""
    <title>訂閱成功</title>
    <div>
        <span style="width:300px;height:100px;text-align:center;line-height:100px;font-size:1.2cm;">感謝你使用訂閱功能</span>
    </div>
    <div>
        <span style="width:300px;height:100px;text-align:center;line-height:100px;font-size:1.2cm;">已訂閱:%s</span>
    </div>
    """%(str(subscription))

    return sub
@app.route('/subscription/search/<userid>',methods= ['GET'])
def sub_search(userid):
    conn = pymysql.connect(host=cloud_ip,
                             port=3306,
                             user=sql_user,
                             password=sql_password,
                             db='news',
                             charset='utf8')

    # 使用 cursor() 方法创建一个游标对象 cursor
    cursor = conn.cursor()
    # 使用 execute()  記錄使用者訂閱
    cursor.execute("select keyword \
                    from user_subscription \
                    where userid = '%s'"%(str(userid))
                    )
    data = cursor.fetchone()
    conn.commit()
    conn.close()
    try:
        result ="""
    <title>訂閱內容</title>
    <div>
        <span style="width:300px;height:100px;text-align:center;line-height:100px;font-size:1.5cm;">您的訂閱內容:%s</span>
    </div>
    """%(str(data[0]))
        
       
    except TypeError:
        result ="""
    <title>訂閱內容</title>
    <div>
        <span style="width:300px;height:100px;text-align:center;line-height:100px;font-size:1.5cm;">還沒有任何訂閱內容</span>
    </div>
    """
    return result

<font face="黑" size=5 color="#000000" >被動傳送訊息規則

In [ ]:
'''

當用戶發出文字消息時，判斷文字內容是否包含::text:，
    若有，則從template_message_dict 內找出相關訊息
    若無，則回傳預設訊息。

'''
# 引用新聞推薦模板 news_recommend  國內新聞(recommend),國外新聞(foreign_recommend)
from news_recommend import (
    recommend,foreign_recommend
)
# # 引用關鍵字模板
from keyword_search import keyword
from linebot.models import(
    FlexSendMessage,CarouselContainer,BubbleContainer
)

# 用來將過多的Liff刪除
from liff_api import *
# initiate token
liff_init(secretFileContentJson.get("channel_access_token"))

# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    userid = user_profile.user_id
#     print(userid)
       # to remove leftover liff
    try:
        if len(liff_list()['apps']) >= 15:
            for i in liff_list()['apps']:
                liff_delete(i['liffId'])
    except:
        print('Liffs have been all removed.')
#     print(event.message.text)
    if(event.message.text.find('[::text:]關於我們')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text = "開發者:\n吳冠廷\n陳俊龍\n王楷欣\n陳重叡\n胡元亨\n郝學偉"))
    elif(event.message.text.find('[::flex:]新聞推薦')!= -1):
        carouselContent = CarouselContainer.new_from_json_dict(json.loads(recommend()))
        flexCarouselSendMeesage =  FlexSendMessage(alt_text="hello", contents=carouselContent)
        line_bot_api.reply_message(
        event.reply_token,
        flexCarouselSendMeesage
        )
    elif(event.message.text.find('[::flex:]國外熱門新聞')!= -1):
        carouselContent = CarouselContainer.new_from_json_dict(json.loads(foreign_recommend()))
        flexCarouselSendMeesage =  FlexSendMessage(alt_text="foreign", contents=carouselContent)
        line_bot_api.reply_message(
        event.reply_token,
        flexCarouselSendMeesage
        )
    elif(event.message.text.find('[::text:]關鍵字搜尋')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        keywd(userid)
        )
    elif(event.message.text.find('[::text:]訂閱')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        subscription(userid)
        )
    elif(event.message.text.find('[::text:]關聯字字典')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        relevance(userid)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="請點擊選單了解更多"))

<font face="黑" size=5 color="#000000" >收集使用者喜好 Postback

In [ ]:
# 用戶點擊button後，觸發postback event，對其回傳做相對應處理
# 收集用戶喜歡新聞類型
from linebot.models import (
     PostbackEvent
)
import json
from kafkaProducer import toKafka
@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
#     print(event)
    news_url = event.postback.data.split("|")[0]
    userid = user_profile.user_id
    preference = event.postback.data.split("|")[1]
    timestamp = event.timestamp
    print('{"news_url":"%s","userid":"%s","preference":"%s","timestamp":"%s"}'%(news_url,userid,preference,timestamp))
    user_preference = '{"news_url":"%s","userid":"%s","preference":"%s","timestamp":"%s"}'%(news_url,userid,preference,timestamp)
    toKafka("preference","None",user_preference)
    with open("../user_preference.txt","a") as myfile:
        myfile.write(user_preference)
        myfile.write('\r\n')
        
    if (preference == 'like'):
        line_bot_api.reply_message(
        event.reply_token,
            TextMessage(text='喜歡的話之後會增加此類型出現')
        )
    elif (preference == 'dislike'):
        line_bot_api.reply_message(
        event.reply_token,
            TextMessage(text='不喜歡的話之後會減少此類型出現')
        )
    else:
        pass

In [ ]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


Liffs have been all removed.


172.19.0.2 - - [26/Mar/2019 05:05:43] "POST / HTTP/1.1" 200 -


Liffs have been all removed.


172.19.0.2 - - [26/Mar/2019 05:05:44] "POST / HTTP/1.1" 200 -


...

172.19.0.2 - - [26/Mar/2019 05:05:51] "POST / HTTP/1.1" 200 -


Liffs have been all removed.


172.19.0.2 - - [26/Mar/2019 05:06:54] "POST / HTTP/1.1" 200 -


Liffs have been all removed.


172.19.0.2 - - [26/Mar/2019 05:32:37] "POST / HTTP/1.1" 200 -
172.19.0.2 - - [26/Mar/2019 05:33:51] "POST / HTTP/1.1" 200 -
172.19.0.2 - - [26/Mar/2019 05:34:10] "GET /relevance/U93eab7e9403ce03748e6079ce7c0fa29 HTTP/1.1" 200 -
103.11.244.50 - - [26/Mar/2019 05:44:36] "GET /mysite/wp-login.php HTTP/1.1" 404 -
37.235.170.1 - - [26/Mar/2019 06:49:44] "GET / HTTP/1.1" 405 -
132.232.63.23 - - [26/Mar/2019 07:15:25] "PROPFIND / HTTP/1.1" 405 -
132.232.63.23 - - [26/Mar/2019 07:15:25] "GET /webdav/ HTTP/1.1" 404 -
216.245.197.254 - - [26/Mar/2019 07:24:43] "HEAD /robots.txt HTTP/1.0" 404 -
91.235.175.131 - - [26/Mar/2019 07:32:35] "GET / HTTP/1.1" 405 -
184.107.218.106 - - [26/Mar/2019 07:52:57] "GET /2019/wp-login.php HTTP/1.1" 404 -
138.197.150.253 - - [26/Mar/2019 09:58:20] "GET /2018/wp-login.php HTTP/1.1" 404 -
185.248.100.89 - - [26/Mar/2019 10:11:00] "GET /requested.html HTTP/1.0" 404 -
111.206.52.124 - - [26/Mar/2019 10:29:12] "GET / HTTP/1.1" 405 -
54.37.235.43 - - [26/Mar/2019 10:4